<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-case-studies/blob/master/huggingface-transformers-practice/transformers-use-cases/4_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text Generation

In text generation (a.k.a open-ended text generation) the goal is to create a coherent portion of text that is a continuation from the given context. The following example shows how GPT-2 can be used in pipelines to generate text. As a default all models apply Top-K sampling when used in pipelines, as configured in their respective configurations.

Referemce: https://huggingface.co/transformers/task_summary.html

## Setup

In [ ]:
%tensorflow_version 2.x     # magic command instructing to use TensorFlow version 2+
import tensorflow as tf
import torch

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelWithLMHead, top_k_top_p_filtering
from torch.nn import functional as F
from transformers import TFAutoModelWithLMHead, tf_top_k_top_p_filtering

from pprint import pprint

## Loading Pipeline

Here, the model generates a random text with a total maximal length of 50 tokens from context “As far as I am concerned, I will”.

In [ ]:
text_generator = pipeline("text-generation")

print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=False))

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'As far as I am concerned, I will be the first to admit that I am not a fan of the idea of a "free market." I think that the idea of a free market is a bit of a stretch. I think that the idea'}]


The default arguments of `PreTrainedModel.generate()` can be directly overridden in the pipeline, as is shown above for the argument `max_length`.

Here is an example of text generation using XLNet and its tokenizer.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
model = TFAutoModelWithLMHead.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """
In 1991, the remains of Russian Tsar Nicholas II and his family (except for Alexei and Maria) are discovered.
 The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the remainder of the story. 1883 Western Siberia,
 a young Grigori Rasputin is asked by his father and a group of men to perform magic.
 Rasputin has a vision and denounces one of the men as a horse thief. Although his
 father initially slaps him for making such an accusation, Rasputin watches as the
 man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
 the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
 with people, even a bishop, begging for his blessing. <eod> </s> <eos>
"""

In [ ]:
prompt = "Today the weather is really nice and I am planning on "

inputs = tokenizer.encode(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="tf")

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]

print(generated)

Today the weather is really nice and I am planning on anning on sleeping in a new home. This week, I plan to try out new places and try out new things for myself. As far as I can tell, it is going to be a great beginning to my new year.<eop> In January, my last post was in my Life Journal, for a few weeks after I graduated from college. I said I wanted to post in


Text generation is currently possible with GPT-2, OpenAi-GPT, CTRL, XLNet, Transfo-XL and Reformer in PyTorch and for most models in Tensorflow as well. As can be seen in the example above XLNet and Transfo-XL often need to be padded to work well. GPT-2 is usually a good choice for open-ended text generation because it was trained on millions of webpages with a causal language modeling objective.

For more information on how to apply different decoding strategies for text generation, please also refer to [our text generation blog post here](https://huggingface.co/blog/how-to-generate).

## PyTorch implementation

In [ ]:
# Pytorch
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
model = AutoModelWithLMHead.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """
In 1991, the remains of Russian Tsar Nicholas II and his family (except for Alexei and Maria) are discovered.
 The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the remainder of the story. 1883 Western Siberia,
 a young Grigori Rasputin is asked by his father and a group of men to perform magic.
 Rasputin has a vision and denounces one of the men as a horse thief. Although his
 father initially slaps him for making such an accusation, Rasputin watches as the
 man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
 the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
 with people, even a bishop, begging for his blessing. <eod> </s> <eos>
"""

In [ ]:
prompt = "Today the weather is really nice and I am planning on "

inputs = tokenizer.encode(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]

print(generated)

Today the weather is really nice and I am planning on anning on going to the beach at the beach (Fl. 25 and 26) with my friend A. I can still do this but s... I must get ready and be up in the morning by 2AM!<eop> A.: "I am tired and I am really not good." I "I will get ready today in the morning
